<a href="https://colab.research.google.com/github/amalqahtani/stress_prediction/blob/main/train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Install packages
!pip install transformers
!pip install datasets


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.9 MB 5.2 MB/s 
     |████████████████████████████████| 6.6 MB 57.6 MB/s 
     |████████████████████████████████| 163 kB 63.1 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 431 kB 4.9 MB/s 
     |████████████████████████████████| 115 kB 62.9 MB/s 
     |████████████████████████████████| 212 kB 69.4 MB/s 
     |████████████████████████████████| 127 kB 62.9 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [3]:
#!pip install pytorch
!pip install torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:

import datasets
from datasets import load_dataset

data_files = {"train": "df_train.csv", "test": "df_val.csv"}
train_ds, test_ds = load_dataset('csv', data_files=data_files,split=['train', 'test'])

train_ds, test_ds


Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

0 tables [00:00, ? tables/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-2b16faeb765c42ce/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

(Dataset({
     features: ['text', 'tweet_id', 'label'],
     num_rows: 2936
 }), Dataset({
     features: ['text', 'tweet_id', 'label'],
     num_rows: 420
 }))

In [6]:
labels_train = train_ds['label']
labels_test = test_ds['label']

#print(labels_test)
#print(labels_train[10])

In [7]:
"""## Preprocess:

"""

from transformers import (AutoTokenizer, 
                          AutoModelForSequenceClassification, 
                          Trainer, 
                          TrainingArguments)

tokenizer = AutoTokenizer.from_pretrained("distilroberta-base")
tokenize_func = lambda sentences: tokenizer(sentences['text'], \
                                            padding="max_length", \
                                            truncation=True, \
                                            max_length=280)

tok_train_ds = train_ds.map(tokenize_func, batched=True)
tok_test_ds = test_ds.map(tokenize_func, batched=True)


Downloading:   0%|          | 0.00/480 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

  0%|          | 0/3 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [8]:
model = AutoModelForSequenceClassification.from_pretrained("distilroberta-base", num_labels=2)


Downloading:   0%|          | 0.00/331M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilroberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.dense.bias', 'roberta.pooler.dense.weight', 'lm_head.bias', 'lm_head.decoder.weight', 'roberta.pooler.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at distilroberta-base and are newly initialized: ['classifier.out_proj.weig

In [38]:
training_args = TrainingArguments(
    output_dir='./content/distilroberta-base-results',          # output directory
    learning_rate=2e-5,
    num_train_epochs=5,              # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=16,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=500,
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [39]:
from sklearn.metrics import accuracy_score, classification_report,f1_score,precision_score,recall_score
from sklearn.metrics import precision_recall_fscore_support

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='micro')
    acc = accuracy_score(labels, preds)
    report =classification_report(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall,
        'classification_report': report
    }
        #f1_micro = f1_score(labels,preds,average='micro')
        #precision_micro = precision_score(labels, preds,average='micro')
        #recall_micro = recall_score(labels, preds,average='micro')
        #'f1_micro':f1_micro,
        #'recall_micro':recall_micro,
        #'precision_micro':precision_micro,

In [40]:
trainer = Trainer(
    model=model,                         # the instantiated Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=tok_train_ds,         # training dataset
    eval_dataset=tok_test_ds,          # evaluation dataset
    compute_metrics=compute_metrics,     # the callback that computes metrics of interest
)

In [41]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: tweet_id, text. If tweet_id, text are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 2936
  Num Epochs = 6
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 1104


Step,Training Loss
500,0.008600
1000,0.019800


Saving model checkpoint to ./content/distilroberta-base-results/checkpoint-500
Configuration saved in ./content/distilroberta-base-results/checkpoint-500/config.json
Model weights saved in ./content/distilroberta-base-results/checkpoint-500/pytorch_model.bin
Saving model checkpoint to ./content/distilroberta-base-results/checkpoint-1000
Configuration saved in ./content/distilroberta-base-results/checkpoint-1000/config.json
Model weights saved in ./content/distilroberta-base-results/checkpoint-1000/pytorch_model.bin


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=1104, training_loss=0.013603690849698109, metrics={'train_runtime': 2636.2179, 'train_samples_per_second': 6.682, 'train_steps_per_second': 0.419, 'total_flos': 1276157801802240.0, 'train_loss': 0.013603690849698109, 'epoch': 6.0})

In [43]:

#print("---Training Evaluation----------")
print(trainer.evaluate())


The following columns in the evaluation set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: tweet_id, text. If tweet_id, text are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 420
  Batch size = 16


Trainer is attempting to log a value of "              precision    recall  f1-score   support

           0       0.87      0.79      0.83       264
           1       0.69      0.81      0.75       156

    accuracy                           0.80       420
   macro avg       0.78      0.80      0.79       420
weighted avg       0.81      0.80      0.80       420
" of type <class 'str'> for key "eval/classification_report" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.


{'eval_loss': 1.765789270401001, 'eval_accuracy': 0.7952380952380952, 'eval_f1': 0.7952380952380952, 'eval_precision': 0.7952380952380952, 'eval_recall': 0.7952380952380952, 'eval_classification_report': '              precision    recall  f1-score   support\n\n           0       0.87      0.79      0.83       264\n           1       0.69      0.81      0.75       156\n\n    accuracy                           0.80       420\n   macro avg       0.78      0.80      0.79       420\nweighted avg       0.81      0.80      0.80       420\n', 'eval_runtime': 18.4763, 'eval_samples_per_second': 22.732, 'eval_steps_per_second': 1.461, 'epoch': 6.0}


In [44]:
trainer.save_model()
tokenizer.save_pretrained("./content/distilroberta-base-results")

Saving model checkpoint to ./content/distilroberta-base-results
Configuration saved in ./content/distilroberta-base-results/config.json
Model weights saved in ./content/distilroberta-base-results/pytorch_model.bin
tokenizer config file saved in ./content/distilroberta-base-results/tokenizer_config.json
Special tokens file saved in ./content/distilroberta-base-results/special_tokens_map.json


('./content/distilroberta-base-results/tokenizer_config.json',
 './content/distilroberta-base-results/special_tokens_map.json',
 './content/distilroberta-base-results/vocab.json',
 './content/distilroberta-base-results/merges.txt',
 './content/distilroberta-base-results/added_tokens.json',
 './content/distilroberta-base-results/tokenizer.json')